In [1]:
import os
import yaml
import torch
import torch.nn as nn
from codes.utils.device import device
from codes.models.FNO import TensorizedFNO
from codes.models.CNO import CompressedCNO
from codes.data.dataset import LidDrivenDataset
from codes.utils.visualization import plot_ldc_like

Using CUDA for tensor operations


In [2]:
def read_yaml_file(folder_path, file_name):
    """Reads a YAML file and returns the data."""
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")
            return None

In [4]:
def all_metrics_validation_set(y, yhat, resolution):

  half_res = int(resolution/2)

  mse = nn.MSELoss()

  mses = torch.zeros(7) # we are outputing 7 validation metrics

  #mse full
  mses[0] = mse(y,yhat)

  #mse u,v
  mses[1] = mse(y[:,0:2,:,:], yhat[:,0:2,:,:])

  #mse p
  mses[2] = mse(y[:,2,:,:], yhat[:,2,:,:])

  #mse near object u, v
  if resolution == 128:
    mses[3] = mse(y[:,0:2,25:104,25:104], yhat[:,0:2,25:104,25:104])
  elif resolution == 256:
    mses[3] = mse(y[:,0:2,51:206,51:206], yhat[:,0:2,51:206,51:206])
  else:
    mses[3] = mse(y[:,0:2,102:410,102:410], yhat[:,0:2,102:410,102:410])

  #mse near object p
  if resolution == 128:
    mses[4] = mse(y[:,2,25:104,25:104], yhat[:,2,25:104,25:104])
  elif resolution == 256:
    mses[4] = mse(y[:,2,51:206,51:206], yhat[:,2,51:206,51:206])
  else:
    mses[4] = mse(y[:,2,102:410,102:410], yhat[:,2,102:410,102:410])

  #mse cd
  mses[5] = mse(y[:,3,:half_res,:], yhat[:,3,:half_res,:])

  #mse cl
  mses[6] = mse(y[:,3,half_res:,:], yhat[:,3,half_res:,:])
  
  #return
  return mses

In [9]:
dataset = LidDrivenDataset(file_path_x= '../NS/512x512/harmonics_lid_driven_cavity_X.npz', 
                           file_path_y = '../NS/512x512/harmonics_lid_driven_cavity_Y.npz')
train_loader, val_loader = dataset.create_dataloader(batch_size= 5, split_fraction= 0.1, shuffle=True)

In [12]:
torch.cuda.empty_cache()
config_data = read_yaml_file('experiments/cno_1', "config.yaml")

CNO_trained_1 = CompressedCNO(in_dim = config_data['in_dim'], out_dim = config_data['out_dim'], 
                          N_layers = config_data['N_layers'], in_size = config_data['in_size'], 
                          out_size = config_data['out_size']).to(device)

CNO_trained_1.load_checkpoint(save_name="400", save_folder='experiments/cno_1/checkpoints')

mse_0 = 0
mse_1 = 0
mse_2 = 0
mse_3 = 0
mse_4 = 0
mse_5 = 0
mse_6 = 0

CNO_trained_1.eval()
with torch.no_grad():
    for batch in val_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        outputs = CNO_trained_1.forward(inputs)

        mses = all_metrics_validation_set(targets, outputs, 512)
        mse_0 += mses[0]
        mse_1 += mses[1]
        mse_2 += mses[2]
        mse_3 += mses[3]
        mse_4 += mses[4]
        mse_5 += mses[5]
        mse_6 += mses[6]

Setting up PyTorch plugin "filtered_lrelu_plugin"... 

/ext3/miniconda3/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.


In [4]:
config_data = read_yaml_file('experiments/fno_1/', "config.yaml")
FNO_trained_1 = TensorizedFNO(n_modes = config_data['n_modes'], in_channels = config_data['in_channels'], 
                              out_channels = config_data['out_channels'], hidden_channels = config_data['hidden_channels'], 
                              projection_channels = config_data['projection_channels'], n_layers = config_data['n_layers']).to(device)

FNO_trained_1.load_checkpoint(save_name="200", save_folder='experiments/fno_1/checkpoints')

FNO_trained_1.eval()
with torch.no_grad():
    
    for idx in range(len(dataset)):
        if idx>50:
            break

        id = 2999 - idx
        inputs, targets = dataset[id]
        outputs = FNO_trained_1.forward(inputs.unsqueeze(0).to(device))

        plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), outputs.cpu().numpy(), 0, os.path.join('experiments/fno_1/plots', f'FNO_1_{id}.png'))

/scratch/au2216/projects/FlowBench/codes/utils/visualization.py:57: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [5]:
config_data = read_yaml_file('experiments/fno_2/', "config.yaml")
FNO_trained_2 = TensorizedFNO(n_modes = config_data['n_modes'], in_channels = config_data['in_channels'], 
                              out_channels = config_data['out_channels'], hidden_channels = config_data['hidden_channels'], 
                              projection_channels = config_data['projection_channels'], n_layers = config_data['n_layers']).to(device)

FNO_trained_2.load_checkpoint(save_name="200", save_folder='experiments/fno_2/checkpoints')

FNO_trained_2.eval()
with torch.no_grad():
    
    for idx in range(len(dataset)):
        if idx>50:
            break
            
        id = 2999 - idx
        inputs, targets = dataset[id]
        outputs = FNO_trained_2.forward(inputs.unsqueeze(0).to(device))
        
        plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), outputs.cpu().numpy(), 0, os.path.join('experiments/fno_2/plots', f'FNO_2_{id}.png'))

In [6]:
torch.cuda.empty_cache()

In [7]:
config_data = read_yaml_file('experiments/cno_1/', "config.yaml")

CNO_trained_1 = CompressedCNO(in_dim = config_data['in_dim'], out_dim = config_data['out_dim'], 
                          N_layers = config_data['N_layers'], in_size = config_data['in_size'], 
                          out_size = config_data['out_size']).to(device)

CNO_trained_1.load_checkpoint(save_name="800", save_folder='experiments/cno_1/checkpoints')

CNO_trained_1.eval()
with torch.no_grad():
    
    for idx in range(len(dataset)):
        if idx>50:
            break

        id = 2999 - idx
        inputs, targets = dataset[id]
        outputs = CNO_trained_1.forward(inputs.unsqueeze(0).to(device))
        
        plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), outputs.cpu().numpy(), 0, os.path.join('experiments/cno_1/plots', f'CNO_1_{id}.png'))

Setting up PyTorch plugin "filtered_lrelu_plugin"... 

/ext3/miniconda3/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.


/scratch/au2216/projects/FlowBench/codes/utils/visualization.py:57: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [8]:
config_data = read_yaml_file('experiments/cno_2/', "config.yaml")

CNO_trained_2 = CompressedCNO(in_dim = config_data['in_dim'], out_dim = config_data['out_dim'], 
                          N_layers = config_data['N_layers'], in_size = config_data['in_size'], 
                          out_size = config_data['out_size']).to(device)

CNO_trained_2.load_checkpoint(save_name="400", save_folder='experiments/cno_2/checkpoints')

CNO_trained_2.eval()
with torch.no_grad():
    
    for idx in range(len(dataset)):
        if idx>50:
            break

        id = 2999 - idx
        inputs, targets = dataset[id]
        outputs = CNO_trained_2.forward(inputs.unsqueeze(0).to(device))
        
        plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), outputs.cpu().numpy(), 0, os.path.join('experiments/cno_2/plots', f'CNO_2_{id}.png'))

In [9]:
!ls experiments/fno_1/checkpoints

200.pth


In [10]:
!ls experiments/fno_2/checkpoints

200.pth


In [11]:
!ls experiments/cno_1/checkpoints

200.pth  400.pth  600.pth  800.pth


In [12]:
!ls experiments/cno_2/checkpoints

200.pth  400.pth
